# jma-maxtemp-00-STEP2-merge-historical_data_by_points


* 作業内容
    * jma-maxtemp-00-STEP1-get-historical_data_by_pointsで取得した各観測時点のヒストリカルデータをまとめる
    * 県別の日次の最高気温の推移を作成
    * データ内容（欠損値）をチェック

* 不定期運転
    * 最初のデータ作成時
    * STEP1で過去データを遡及して取得して期間をのばしたとき
    * 月次データの更新など（やりたければ）
<br><br>
* 次のステップ
    * 最新のデータと統合して、描画用のデータを作る

In [1]:
import pandas as pd
import glob
import re
from itertools import product

観測地点

In [2]:
#観測地点の一覧
url = 'https://raw.githubusercontent.com/Nikkei-Visual-Data-Journalism/Heatwave/main/data-maxtemp/meta/points_list.csv'
points = pd.read_csv(url)

In [3]:
#地点、日付の組み合わせ
prec_no = set(points.prec_no)
dates = pd.date_range('1950-01-01','2023-07-01',freq='MS')

取得済みのデータを呼び出し

In [4]:
#観測地点ごとの過去データ
file_dir = "./data-maxtemp/timeseries-data-by-points/"

In [5]:
#ダウンロード済みファイル
file_list= glob.glob(f'{file_dir}data-raw/prec-*/**/*.csv', recursive=True)

In [6]:
pattern = r"/prec-(\d+)/jma-maxtemp-hs-\d+-(\d+)\.csv$"
data_list = []

for f in file_list:
    prec, yyyymm = re.search(pattern, f).groups()
    data =  {'prec_no': int(prec), 'yyyymm':yyyymm,'data':1,'filepath':f}
    data_list.append(data)
    
retrieved = pd.DataFrame(data_list)
retrieved.yyyymm = pd.to_datetime(retrieved.yyyymm, format='%Y%m')

In [7]:
retrieved.head()

,prec_no,yyyymm,data,filepath
0,64,2010-02-01,1,./data-maxtemp/timeseries-data-by-points/data-...
1,64,2003-08-01,1,./data-maxtemp/timeseries-data-by-points/data-...
2,64,2016-10-01,1,./data-maxtemp/timeseries-data-by-points/data-...
3,64,2016-04-01,1,./data-maxtemp/timeseries-data-by-points/data-...
4,64,2009-03-01,1,./data-maxtemp/timeseries-data-by-points/data-...


データを統合

In [8]:
data_all = pd.DataFrame()

for prec in prec_no:
    filepaths = retrieved[retrieved.prec_no==prec].dropna(subset='filepath').filepath.to_list()
    data_agg = pd.DataFrame()

    for filepath in filepaths:
        data_monthly = pd.read_csv(filepath)     
        #)や]が入っているので掃除
        data_monthly.maxtemp = data_monthly.maxtemp.apply(lambda x: re.sub(r'[^\d\.-]', '', str(x)))
        data_monthly.maxtemp = pd.to_numeric(data_monthly.maxtemp, errors='coerce').astype(float)
        #まとめる
        data_agg = pd.concat([data_agg, data_monthly])
        #都道府県情報
        pref_dic = points.set_index('prec_no').pref.to_dict()
        data_agg['pref'] = data_agg.prec_no.map(pref_dic)
        #都道府県庁所在地
        capitol = points[points.capitol==1]['観測所番号'].to_list()
        data_agg['capitol'] = None
        data_agg.loc[data_agg.points_no.isin(capitol),'capitol'] = 1
        #日付、年
        data_agg.date = pd.to_datetime(data_agg.date)
        data_agg['year'] = data_agg.date.dt.year
    #地点ごとのデータを出力
    output_dir = f'{file_dir}data-agg-by-points/jma-maxtemp-hs-{prec}-merged.csv'
    data_agg.to_csv(output_dir, index=False)
    print(f'finished: {prec}')
        
    #１つのファイルに統合
    data_all = pd.concat([data_all,data_agg])
    
#output_dir = f'{file_dir}data-agg-by-points/jma-maxtemp-hs-all-merged.csv'
#data_all.to_csv(output_dir, index=False)

finished: 11
finished: 12
finished: 13
finished: 14
finished: 15
finished: 16
finished: 17
finished: 18
finished: 19
finished: 20
finished: 21
finished: 22
finished: 23
finished: 24
finished: 31
finished: 32
finished: 33
finished: 34
finished: 35
finished: 36
finished: 40
finished: 41
finished: 42
finished: 43
finished: 44
finished: 45
finished: 46
finished: 48
finished: 49
finished: 50
finished: 51
finished: 52
finished: 53
finished: 54
finished: 55
finished: 56
finished: 57
finished: 60
finished: 61
finished: 62
finished: 63
finished: 64
finished: 65
finished: 66
finished: 67
finished: 68
finished: 69
finished: 71
finished: 72
finished: 73
finished: 74
finished: 81
finished: 82
finished: 83
finished: 84
finished: 85
finished: 86
finished: 87
finished: 88
finished: 91


真夏日、猛暑日を計算

In [9]:
#県内の最高温度
df_count = data_all.groupby(['date','year','pref']).maxtemp.max()

In [10]:
#県庁所在地の最高温度をつけたす
capitol = data_all[data_all.capitol==1].set_index(['date','year','pref']).maxtemp.rename('maxtemp_capitol')
df_count = pd.concat([df_count, capitol],axis=1)

In [11]:
#フラグ
over30 = (df_count >=30).add_prefix('over30_')
over35 = (df_count >=35).add_prefix('over35_')
over40 = (df_count >=40).add_prefix('over40_')

In [12]:
df_count = pd.concat([df_count, over30, over35, over40],axis=1)

In [13]:
df_count.columns = df_count.columns.str.replace('_maxtemp', '', regex=False)

In [15]:
df_count = df_count.reset_index()

In [16]:
df_count

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol,count
0,2000-01-01,2000,三重県,14.6,11.7,False,False,False,False,False,False,1
1,2000-01-01,2000,京都府,9.6,9.3,False,False,False,False,False,False,1
2,2000-01-01,2000,佐賀県,16.9,16.3,False,False,False,False,False,False,1
3,2000-01-01,2000,兵庫県,12.5,11.2,False,False,False,False,False,False,1
4,2000-01-01,2000,北海道,2.7,-0.7,False,False,False,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...
404806,2023-07-31,2023,静岡県,35.9,32.8,True,True,True,False,False,False,1
404807,2023-07-31,2023,香川県,34.5,33.0,True,True,False,False,False,False,1
404808,2023-07-31,2023,高知県,32.7,32.7,True,True,False,False,False,False,1
404809,2023-07-31,2023,鳥取県,36.9,36.9,True,True,True,True,False,False,1


直近データと統合

In [56]:
#直近データを呼び出し
filename = './data-maxtemp/timeseries-data/jma-maxtemp-table-ts.csv'
df_count_latest = pd.read_csv(filename)

In [57]:
df_count_merged = pd.concat([df_count, df_count_latest])

In [58]:
df_count_merged.date = pd.to_datetime(df_count_merged.date)

In [59]:
df_count_merged = df_count_merged[~df_count_merged.duplicated(subset=['date','pref'],keep='first')].reset_index(drop=True)

In [45]:
file_dir = "./data-maxtemp/timeseries-data/jma-maxtemp-temp-by-pref-ts.csv"

In [60]:
df_count_merged.to_csv(file_dir, index=False)

In [61]:
df_count_merged

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol
0,2000-01-01,2000,三重県,14.6,11.7,False,False,False,False,False,False
1,2000-01-01,2000,京都府,9.6,9.3,False,False,False,False,False,False
2,2000-01-01,2000,佐賀県,16.9,16.3,False,False,False,False,False,False
3,2000-01-01,2000,兵庫県,12.5,11.2,False,False,False,False,False,False
4,2000-01-01,2000,北海道,2.7,-0.7,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
405135,2023-08-07,2023,静岡県,33.8,33.4,True,True,False,False,False,False
405136,2023-08-07,2023,香川県,29.5,29.5,False,False,False,False,False,False
405137,2023-08-07,2023,高知県,30.9,30.0,True,True,False,False,False,False
405138,2023-08-07,2023,鳥取県,34.8,34.3,True,True,False,False,False,False


データ内容をチェック

In [62]:
#データ漏れのチェック
test = df_count_merged.pivot(index='date',columns='pref',values='maxtemp_capitol')
test = test.iloc[:-3].fillna('no_data')
no_data = test.where(test == 'no_data').dropna(how='all').dropna(how='all',axis=1)
no_data = no_data.unstack().dropna().rename('maxtemp_capitol').reset_index()
no_data['prec_no'] = no_data.pref.map(points.set_index('pref').prec_no.to_dict())
no_data

,pref,date,maxtemp_capitol,prec_no
0,兵庫県,2000-01-24,no_data,63
1,埼玉県,2001-03-18,no_data,43
2,奈良県,2000-02-09,no_data,64
3,富山県,2003-11-21,no_data,55
4,岡山県,2000-08-22,no_data,66
5,岡山県,2001-09-28,no_data,66
6,広島県,2001-02-27,no_data,67
7,新潟県,2003-10-30,no_data,54
8,新潟県,2004-07-07,no_data,54
9,香川県,2002-02-20,no_data,72


In [63]:
#データ欠損分の元データをたどって表示
no_data_raw = pd.DataFrame()

for idx, row in no_data.iterrows():
    pref = row['pref']
    date = row['date']
    prec_no = row['prec_no']
    #filepath
    yyyymm = pd.to_datetime(date).strftime('%Y%m')
    filepath = f"./data-maxtemp/timeseries-data-by-points/data-raw/prec-{prec_no}/jma-maxtemp-hs-{prec_no}-{yyyymm}.csv"
    #data by points
    data = pd.read_csv(filepath) 
    capitol = points[(points.capitol==1)&(points.prec_no==prec_no)].name.values[0]
    data = data[(data.name==capitol)&(data.date==date.strftime('%Y-%m-%d'))]
    no_data_raw = pd.concat([no_data_raw, data])

In [64]:
#元データがもともと欠損しているので、欠損のままでOK
no_data_raw

,name,date,maxtemp,prec_no,points_no
333,神戸,2000-01-24,×,63,63518.0
110,さいたま,2001-03-18,///,43,43241.0
8,奈良,2000-02-09,×,64,64036.0
110,富山,2003-11-21,×,55,55102.0
207,岡山,2000-08-22,×,66,66408.0
207,岡山,2001-09-28,×,66,66408.0
222,広島,2001-02-27,×,67,67437.0
122,新潟,2003-10-30,×,54,54232.0
99,新潟,2004-07-07,×,54,54232.0
47,高松,2002-02-20,×,72,72086.0
